In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo

# Establish nongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.biz_db

# Declare the collection
bikes = db.bikes

# Drop the collection
bikes.drop()

url = 'https://newyork.craigslist.org/search/bia?query=bicycle&sort=rel'
response = requests.get(url)

# Get HTML code behind the page
# Different parser options: html.parser, lxml, xml, lxml-xml, html5lib
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser
soup = BeautifulSoup(response.text, 'lxml')
# print(soup.prettify())

# Set outputs to a list of all product information with li tag
outputs = soup.find_all('li', class_='result-row')

# For each product, get title, price and url
for out in outputs:
    try:
        title = out.find('a', class_='result-title').text
        price = out.find('span', class_='result-price').text
        #  <span class="result-price">$20</span>

        #         price = out.a.span.text
        #  <a class="result-image gallery" data-ids="3:00000_dhzBRQ6ARaE_0ww0j6,3:00y0y_1DF8txFNuqI_0ww0oo,3:01515_5GJR7p9TiOe_0ww0oo,3:00Y0Y_bDnhFqmD7Qi_0lM0t2,3:00505_4QtOH76pFPK_0ww0oo,3:00d0d_6kvECXrLpMh_0ww0oo,3:00e0e_ctLIGVnVy77_0ww0oo,3:00505_bIDjx1jookI_0ww0oo" href="https://newyork.craigslist.org/mnh/bik/d/new-york-specialized-globe-bicycle/7254301482.html">
        #  <span class="result-price">$650</span>
        #  </a>
        
        # Convert price to int
        price = price.replace('$', '').replace(',', '')
        price = int(price)
        
        # Get the attribute of the element using [], anything other than class
        url = out.a['href']
        
        
        dict_data = {
            'title': title,
            'price': price,
            'url': url
        }
        
        bikes.insert_one(dict_data)
        
    except AttributeError as e:
        print(e)

In [2]:
# Print bikes that are between $50 and $400 sorted by price asc and id desc
bikes_craigs = bikes.find({'price': {'$gt':50, '$lt':200}}).sort([('price', 1), ('_id', -1)])
for bike in bikes_craigs:
    print(bike)
bikes_craigs.count()

{'_id': ObjectId('5fec0dff5c02877566317bd6'), 'title': "Woman's Schwinn Cruiser Bicycle", 'price': 60, 'url': 'https://newyork.craigslist.org/lgi/bik/d/glen-head-womans-schwinn-cruiser-bicycle/7246462798.html'}
{'_id': ObjectId('5fec0dff5c02877566317bfe'), 'title': 'Girls Monster High Bicycle!  EXCELLENT - Best Offer Accepted!', 'price': 71, 'url': 'https://newyork.craigslist.org/brk/bik/d/brooklyn-girls-monster-high-bicycle/7249690932.html'}
{'_id': ObjectId('5fec0dff5c02877566317bef'), 'title': 'Women’s 26” Huffy Trail Runner Bicycle', 'price': 75, 'url': 'https://newyork.craigslist.org/que/bik/d/howard-beach-womens-26-huffy-trail/7249939000.html'}
{'_id': ObjectId('5fec0dff5c02877566317bc5'), 'title': 'Vintage Raleigh Beach Cruiser Bike Bicycle', 'price': 75, 'url': 'https://newyork.craigslist.org/mnh/bik/d/new-york-vintage-raleigh-beach-cruiser/7242646484.html'}
{'_id': ObjectId('5fec0dff5c02877566317ba1'), 'title': 'TREK MYSTIC Bicycle 9 inch frame, 20inch tires', 'price': 80, 'ur

C:\Users\80104697\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


35

In [3]:
bikes.count_documents({})


120

## Web Splinter

#### Interacts with the web browser programmatically
#### Automates browser actions such as filling fields and clicking buttons

In [8]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [9]:
# Drop the collection
bikes.drop()


In [10]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
# Visit url
url = 'https://newyork.craigslist.org/search/bia?query=bicycle&sort=rel'
browser.visit(url)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\80104697\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [11]:
try:
    while True:

        html = browser.html
        soup = BeautifulSoup(html, 'lxml')

        outputs = soup.find_all('li', class_='result-row')

        # For each product, get title, price and url
        for out in outputs:
            try:
                title = out.find('a', class_='result-title').text
                price = out.find('span', class_='result-price').text

                # Convert price to int
                price = price.replace('$', '').replace(',', '')
                price = int(price)

                # Get the attribute of the element using [], anything other than class
                url = out.a['href']


                dict_data = {
                    'title': title,
                    'price': price,
                    'url': url
                }

                bikes.insert_one(dict_data)

            except AttributeError as e:
                print(e)

        browser.links.find_by_partial_text('next').click()
        
except:
    print("Last Page reached")
    
finally:
    print(bikes.count_documents({}))
    browser.quit()


Last Page reached
484
